In [1]:
ROOT_DIR = '/content'

In [4]:
!py -m pip install ultralytics

  Obtaining dependency information for ultralytics from https://files.pythonhosted.org/packages/7b/e0/fec43312f8af41c2bba6aec53c9ff51160e9b10473a603f5388055249b77/ultralytics-8.2.68-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/41.3 kB ? eta -:--:--
     ------------------ ------------------- 20.5/41.3 kB 640.0 kB/s eta 0:00:01
     -------------------------------------  41.0/41.3 kB 653.6 kB/s eta 0:00:01
     -------------------------------------- 41.3/41.3 kB 398.7 kB/s eta 0:00:00
  Obtaining dependency information for opencv-python>=4.6.0 from https://files.pythonhosted.org/packages/ec/6c/fab8113424af5049f85717e8e527ca3773299a3c6b02506e66436e19874f/opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl.metadata
  Obtaining dependency information for torch>=1.8.0 from https://files.pythonhosted.org/packages/18/cf/f69dff972a748e08e1bf602ef94ea5c6d4dd2f41cea22c8ad67a607d8b41/torch-2.4.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for t

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.2.1 -> 24.1.2
[notice] To update, run: C:\Users\coleb\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


# Download Dataset (Images and Labels)

In [5]:
%%bash

file_url="https://drive.google.com/uc?export=download&id=1abdGpzVcLd6SKhP593EKSFkbA6_Mmn7J"
destination_path="/content/wildlife_detector.zip"

confirmation_page=$(curl -s -L "$file_url")

file_id=$(echo "$confirmation_page" | grep -oE "name=\"id\" value=\"[^\"]+" | sed 's/name="id" value="//')
file_confirm=$(echo "$confirmation_page" | grep -oE "name=\"confirm\" value=\"[^\"]+" | sed 's/name="confirm" value="//')
file_uuid=$(echo "$confirmation_page" | grep -oE "name=\"uuid\" value=\"[^\"]+" | sed 's/name="uuid" value="//')

download_url="https://drive.usercontent.google.com/download?id=$file_id&export=download&confirm=$file_confirm&uuid=$file_uuid"

curl -L -o "$destination_path" "$download_url"

if [ $? -eq 0 ]; then
    echo "Download completed successfully."
else
    echo "Download failed."
fi

unzip $destination_path

<3>WSL (9) ERROR: CreateProcessParseCommon:708: Failed to translate C:\Users\coleb\OneDrive\Documents\Gitlab_Repositories\data612-final
<3>WSL (9) ERROR: CreateProcessParseCommon:754: getpwuid(0) failed 2
<3>WSL (9) ERROR: UtilTranslatePathList:2866: Failed to translate C:\Program Files (x86)\VMware\VMware Player\bin\
<3>WSL (9) ERROR: UtilTranslatePathList:2866: Failed to translate C:\Windows\system32
<3>WSL (9) ERROR: UtilTranslatePathList:2866: Failed to translate C:\Windows
<3>WSL (9) ERROR: UtilTranslatePathList:2866: Failed to translate C:\Windows\System32\Wbem
<3>WSL (9) ERROR: UtilTranslatePathList:2866: Failed to translate C:\Windows\System32\WindowsPowerShell\v1.0\
<3>WSL (9) ERROR: UtilTranslatePathList:2866: Failed to translate C:\Windows\System32\OpenSSH\
<3>WSL (9) ERROR: UtilTranslatePathList:2866: Failed to translate C:\Program Files\Git\cmd
<3>WSL (9) ERROR: UtilTranslatePathList:2866: Failed to translate C:\Program Files\MATLAB\R2023b\bin
<3>WSL (9) ERROR: UtilTransla

CalledProcessError: Command 'b'\nfile_url="https://drive.google.com/uc?export=download&id=1abdGpzVcLd6SKhP593EKSFkbA6_Mmn7J"\ndestination_path="/content/wildlife_detector.zip"\n\nconfirmation_page=$(curl -s -L "$file_url")\n\nfile_id=$(echo "$confirmation_page" | grep -oE "name=\\"id\\" value=\\"[^\\"]+" | sed \'s/name="id" value="//\')\nfile_confirm=$(echo "$confirmation_page" | grep -oE "name=\\"confirm\\" value=\\"[^\\"]+" | sed \'s/name="confirm" value="//\')\nfile_uuid=$(echo "$confirmation_page" | grep -oE "name=\\"uuid\\" value=\\"[^\\"]+" | sed \'s/name="uuid" value="//\')\n\ndownload_url="https://drive.usercontent.google.com/download?id=$file_id&export=download&confirm=$file_confirm&uuid=$file_uuid"\n\ncurl -L -o "$destination_path" "$download_url"\n\nif [ $? -eq 0 ]; then\n    echo "Download completed successfully."\nelse\n    echo "Download failed."\nfi\n\nunzip $destination_path\n'' returned non-zero exit status 1.

# Train YOLO Model with Custom Animal Dataset

In [ ]:
import os

from ultralytics import YOLO


# Load a model
# model = YOLO("yolov8n.yaml")  # build a new model from scratch
model = YOLO("yolov8n.pt")  # build a pre-trained model


# Use the model
results = model.train(data=os.path.join(ROOT_DIR, "wildlife_detector", "config.yaml"), epochs=5)  # train the model

# Predict Animal Labels from Animal Videos

In [ ]:
import os

from ultralytics import YOLO
import cv2


VIDEOS_DIR = os.path.join(ROOT_DIR, "wildlife_detector", 'videos')

video_path = os.path.join(VIDEOS_DIR, 'zebra.mp4')
video_path_out = '{}_out.mp4'.format(video_path)

cap = cv2.VideoCapture(video_path)
ret, frame = cap.read()
H, W, _ = frame.shape
out = cv2.VideoWriter(video_path_out, cv2.VideoWriter_fourcc(*'MP4V'), int(cap.get(cv2.CAP_PROP_FPS)), (W, H))

model_path = os.path.join(ROOT_DIR, 'runs', 'detect', 'train', 'weights', 'best.pt')

# Load a model
model = YOLO(model_path)  # load a custom model

threshold = 0.5

while ret:

    results = model(frame)[0]

    for result in results.boxes.data.tolist():
        x1, y1, x2, y2, score, class_id = result

        if score > threshold:
            color = (0, 0, 255)
            if "NOT DANGEROUS" in results.names[int(class_id)].upper():
              color = (0, 255, 0)
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 4)
            cv2.putText(frame, results.names[int(class_id)].upper(), (int(x1), int(y1 - 10)),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.3, color, 3, cv2.LINE_AA)

    out.write(frame)
    ret, frame = cap.read()

cap.release()
out.release()
cv2.destroyAllWindows()